In [1]:
'''

截取所有条件的epoch，然后进行清洗。

截取的长度不一样，RSA和ASSR要分开处理

所有epoch ，然后再来考虑 evoked


RSA 实验

刺激	恒定duration	mark
ba1.wav	1	101
ba2.wav	1	102
ba3.wav	1	103
ba4.wav	1	104
da1.wav	1	111
da2.wav	1	112
da3.wav	1	113
da4.wav	1	114
pa1.wav	1	121
pa2.wav	1	122
pa3.wav	1	123
pa4.wav	1	124
ta1.wav	1	131
ta2.wav	1	132
ta3.wav	1	133
ta4.wav	1	134



ASSR 实验

4_18s.wav	4	4	21.7
6_18s.wav	6	6	21.7

'''


'\n\n截取所有条件的epoch，然后进行清洗。\n\n截取的长度不一样，RSA和ASSR要分开处理\n\n所有epoch ，然后再来考虑 evoked\n\n\nRSA 实验\n\n刺激\t恒定duration\tmark\nba1.wav\t1\t101\nba2.wav\t1\t102\nba3.wav\t1\t103\nba4.wav\t1\t104\nda1.wav\t1\t111\nda2.wav\t1\t112\nda3.wav\t1\t113\nda4.wav\t1\t114\npa1.wav\t1\t121\npa2.wav\t1\t122\npa3.wav\t1\t123\npa4.wav\t1\t124\nta1.wav\t1\t131\nta2.wav\t1\t132\nta3.wav\t1\t133\nta4.wav\t1\t134\n\n\n\nASSR 实验\n\n4_18s.wav\t4\t4\t21.7\n6_18s.wav\t6\t6\t21.7\n\n'

In [2]:
import mne
import os.path as op
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
from autoreject import AutoReject

In [3]:


sub_ids = [

    "pre4008_y","pre4009_y","pre4010_y","pre4011_y","pre4012_y","pre4013_y",
    "pre4014_y","pre4015_y","pre4016_y","pre4017_y", "pre4018_y","pre4019_y",
    "pre4021_y","pre4022_y","pre4023_y","pre4026_y",
    "pre4027_y","pre4028_y","pre4029_y","pre4030_y"

]
# sub_ids = [
#
#     "pre4008_y","pre4009_y",
#
# ]


In [4]:

for sub_id in sub_ids:
    #for sub_id,i in zip(sub_ids,range(len(sub_ids))):
        #读入之前处理好的reconst_raw

    sub_id = sub_ids[0]
    i = 0
    save_path_ica = '.\data\\4raw_ica\\'
    reconst_raw=mne.io.read_raw_fif(save_path_ica + sub_id + 'RSA.fif')

    target_events_RSA = {
                                '101':101,
                                '102':102,
                                '103':103,
                                '104':104,
                                '111':111,
                                '112':112,
                                '113':113,
                                '114':114,
                                '121':121,
                                '122':122,
                                '123':123,
                                '124':124,
                                '131':131,
                                '132':132,
                                '133':133,
                                '134':134,

                         }

    #先找events，epoch和average都是一步的事情。
    custom_mapping = target_events_RSA
    #
    (events_from_annot,
     event_dict) = mne.events_from_annotations(reconst_raw, event_id=custom_mapping)
    #还不如直接对 events_from_annot 进行切片
    #先遍历，把合适的单词筛序出来
    #对应的vocabulary已经列出，然后怎么提取呢？
    events_RSA = events_from_annot
    count_remembered_events = len(events_RSA)
    # 导入MATLAB发现event数量多于epoch数量,你怀疑是重合?
    baseline = - 0.5  # 如果需要做时频相关的分析(包括connectivity)都需要吧把baseline弄长一点.
    epochs_RSA = mne.Epochs(reconst_raw, events_RSA, event_id=event_dict, baseline =(-0.5,0),tmin=baseline, tmax=0.8, preload=True, event_repeated='drop')
    from mne.export import export_epochs
    file_path_save = 'L:\yanglab1_EEG\RSA\data\\6epoch_clean_RSA\\'
    export_epochs(file_path_save + sub_ids[0]+".set", epochs=epochs_RSA, fmt = 'eeglab', overwrite = True)
    #查看所有mark dict
    #events_1 = mne.events_from_annotations(reconst_raw)
    '''

    '''
    # save_path_epoch = '.\data\\5epoch_RSA\\'
    # epochs_RSA.save(save_path_epoch + sub_id + '-epo.fif', overwrite=True)

    save_path_epoch = '.\data\\5epoch_RSA\\'
    epochs_RSA.save(save_path_epoch + sub_id + '0.5base-epo.fif', overwrite=True)
        #用autoreject处理一下
    n_interpolates = np.array([1, 4, 32])
    consensus_percs = np.linspace(0, 0.8, 11)
    picks = mne.pick_types(epochs_RSA.info, meg=False, eeg=True, stim=False, eog=False,
                           include=[], exclude=[])
    ar_RSA = AutoReject(n_interpolates,consensus_percs,picks=picks,
                    thresh_method='random_search',random_state=23)
    ar_RSA.fit(epochs_RSA)
    epochs_clean_RSA = ar_RSA.transform(epochs_RSA)

    '''
    不拆了，全放到一起
    '''
    evoked_original_RSA = list()
    evoked_clean_RSA = list()
    #先average看下结果，对比
    #如果结果不错就可以分别保存下来。
    evoked_original_RSA.append(epochs_RSA.average())#没有autoreject的
    evoked_clean_RSA.append(epochs_clean_RSA.average())
    """
    查看效果，看是否需要回去删除电极
    """
    %matplotlib qt

    fig_original = evoked_original_RSA[i].plot(window_title = sub_id)
    fig_clean = evoked_clean_RSA[i].plot(window_title = sub_id)
    #保存

    save_path_epoch_clean = '.\data\\6epoch_clean_RSA\\'
    epochs_clean_RSA.save(save_path_epoch_clean + sub_id + '0.5base_RSA-epo.fif', overwrite=True)
    save_path_evoked = '.\data\\7evoked_RSA\\'
    #为什么这里加i?因为是list，epochs不是list
    evoked_clean_RSA[i].save(save_path_evoked + sub_id + '0.5base_RSA-ave.fif',overwrite=True)

    """
    查看效果
    """
    %matplotlib qt

    fig_original = evoked_original_RSA[i].plot(window_title = sub_id)
    fig_original.savefig(save_path_evoked+'0.5base-_original_' + sub_id)
    fig_clean = evoked_clean_RSA[i].plot(window_title = sub_id)
    fig_clean.savefig(save_path_evoked+'0.5base-_clean_'+ sub_id)

    # fig_original = evoked_original_RSA_30Hz[i].plot(window_title = sub_id)
    # fig_original.savefig(save_path_evoked+'_original_' + sub_id)
    # fig_clean = evoked_clean_RSA_30Hz[i].plot(window_title = sub_id)
    # fig_clean.savefig(save_path_evoked+'_clean_'+ sub_id)


Opening raw data file .\data\4raw_ica\pre4008_yRSA.fif...


C:\Users\SilverBullet\AppData\Local\Temp\ipykernel_21360\4208668916.py:8: RuntimeWarning: This filename (.\data\4raw_ica\pre4008_yRSA.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  reconst_raw=mne.io.read_raw_fif(save_path_ica + sub_id + 'RSA.fif')


    Range : 1187041 ... 2928463 =   1187.041 ...  2928.463 secs
Ready.
Used Annotations descriptions: ['101', '102', '103', '104', '111', '112', '113', '114', '121', '122', '123', '124', '131', '132', '133', '134']
Not setting metadata
1440 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 1440 events and 1301 original time points ...
162 bad epochs dropped
Overwriting existing file.


C:\Users\SilverBullet\AppData\Roaming\Python\Python310\site-packages\eeglabio\epochs.py:101 EEGLABIO: WARNING: Invalid event latencies, ignored for export.
C:\Users\SilverBullet\AppData\Roaming\Python\Python310\site-packages\eeglabio\epochs.py:110 EEGLABIO: WARNING: Some or all annotations have invalid onsets, discarded for export.


Overwriting existing file.
Running autoreject on ch_type=eeg


...

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]





Estimated consensus=0.24 and n_interpolate=4


  0%|          | Repairing epochs : 0/1278 [00:00<?,       ?it/s]

Dropped 17 epochs: 746, 880, 881, 882, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 1183, 1184


ValueError: Format '5base-_original_pre4008_y' is not supported (supported formats: eps, jpeg, jpg, pdf, pgf, png, ps, raw, rgba, svg, svgz, tif, tiff, webp)